In [2]:
import numpy as np
import random
from tqdm.notebook import tqdm


class Machine:
    
    def __init__(self, vulnerable, percentValue, penetrate):
        self.vulnerable = vulnerable
        self.percentValue = percentValue
        self.penetrate = penetrate
        
    def changeVulnerable(self, vulnerable):
        self.vulnerable = vulnerable
    
    def changeCost(self, percentValue):
        self.percentValue = percentValue
        
    def changePenetrate(self, penetrate):
        self.penetrate = penetrate
        
    def pr(self):
        print(self.vulnerable, self.percentValue, self.penetrate)

        
def printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues):
    print('machine values = ', machineValues, 'vulnerable machine values = ', vulnerableMachineValues, 'important machine values =', importantMachineValues)
#получаем кол-во машин
def GetMachineValues():
    machineValues = random.randint(4, 15)
    return machineValues
#получаем кол-во уязвимых машин 
def GetVulnerableMachineValues(machineValues):
    vulnerableMachineValues = random.randint(1, machineValues)
    return vulnerableMachineValues

#получаем номер главной машины
def GetImportantMachineValues(machineValues):
    importantMachineValues = random.randint(1, machineValues)
    return importantMachineValues

#получаем процент, который занимает главная машина 
def GetPercentValue():
    percentValue = random.randint(30, 60)
    return percentValue

def GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues):
    #пока что сделаем цену cost фиксированной
    #дальше придумаем фукнцию для нее
    machineArray = []
    percentValue = GetPercentValue()
    reversePercent = (100 - percentValue) / (machineValues - 1)
    percentValue = percentValue / 100
    reversePercent = reversePercent / 100
    reversePercent = ("%.2f" % reversePercent)
    for i in range(machineValues):
        if i == (importantMachineValues - 1):
            tmp = Machine(False, percentValue, False)
        else:
            tmp = Machine(False, reversePercent, False)
        machineArray.append(tmp)
    #здесь баг - может 2 и более раз сгенерироваться
    #одинаковое число и поэтому кол-во машин будет неточным
    for i in range(0, vulnerableMachineValues):
        j = random.randint(0, machineValues - 1)
        machineArray[j].changeVulnerable(True)
    return machineArray
#возвращает число уязвимых машин, так как из-за бага
#прошлое число уязвимых != новому
def GetVulnerableMachine(machineValues, machineArray):
    vulnerableMachine = 0
    for i in range(machineValues):
        if machineArray[i].vulnerable == True:
            vulnerableMachine += 1
    return vulnerableMachine

def CostFunction(s, a, Ei):
    try:
        return -(s * Ei * Ei)/(a * Ei - 1)
    except ZeroDivisionError:
        return 0.000001

def UtilityDefender(s, a, Ei, machineValues, machineArray):
    # l - процент понесенных потерь машин
    tmp = 0
    percent = machineValues / 100
    l = 0
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == True:
            l += percent
    for i in range(machineValues):
        tmp += CostFunction(s, a, Ei)
    return - l - tmp

def UtilityAttacker(machineValues, machineArray):
    percent = machineValues / 100
    l = 0
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == True:
            l += percent
    return l
    
def AttackerStrategyProportional(machineArray):
    attackerProportionalEffort = []
    for i in range(len(machineArray)):
        tmp = machineArray[i].percentValue
        attackerProportionalEffort.append(tmp)
    return attackerProportionalEffort
        
def AttackerStrategyHigh():
    attackerHighEffort = 1
    return attackerHighEffort
        
def DefenderStrategyNothing():
    return None
    
def DefenderStrategyProportional(machineArray, Ei):
    defenderProportionalEffort = []
    for i in range(len(machineArray)):
        tmp = float(machineArray[i].percentValue) * Ei
        defenderProportionalEffort.append(tmp)
    return defenderProportionalEffort
    
def DefenderStrategyHigh(Ei):
    defenderHighEffort = Ei
    return defenderHighEffort
#P-атакующий H-защитник
def ComparisonStrategyPH(machineArray, importantMachineValues, Ei):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    defenderHighEffort = DefenderStrategyHigh(Ei)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True:
            if attackerProportionalEffort[i] > machineArray[i].percentValue or attackerProportionalEffort[importantMachineValues - 1] > defenderHighEffort:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 1:
        return True
    else:
        return False
#P-атакующий P-защитник
def ComparisonStrategyPP(machineArray, Ei):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    defenderProportionalEffort = DefenderStrategyProportional(machineArray, Ei)
    count = 0
    for i in range(len(machineArray)):
        if float(attackerProportionalEffort[i]) > defenderProportionalEffort[i]:
            if machineArray[i].vulnerable == True:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#P - атакующий N - защитник
def ComparisonStrategyPN(machineArray):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    #Дописать, так как атакующий может взломать более 1 машины
    #Дописал
    count = 0
    for i in range(len(machineArray)):
        if float(attackerProportionalEffort[i]) > float(machineArray[i].percentValue):
            machineArray[i].changePenetrate(True)
            count += 1
    if count > 0:
        return True
    else:
        return False
#H-атакующий P-защитник
def ComparisonStrategyHP(machineArray, Ei, importantMachineValues):
    attackerHighEffort = AttackerStrategyHigh()
    defenderProportionalEffort = DefenderStrategyProportional(machineArray, Ei)
    if attackerHighEffort > defenderProportionalEffort[importantMachineValues - 1]:
        return True
    else:
        return False
#H-атакующий H-защитник
def ComparsionStrategyHH(machineArray, Ei):
    attackerHighEffort = AttackerStrategyHigh()
    defenderHighEffort = DefenderStrategyHigh(Ei)
    if attackerHighEffort > defenderHighEffort:
        return True
    else:
        return False
#H-Атакующий N-защитник
def ComparsionStrategyHN(machineArray, importantMachineValues):
    attackerHighEffort = AttackerStrategyHigh()
    if float(attackerHighEffort) > machineArray[importantMachineValues - 1].percentValue:
        return True
    else:
        return False

def generateParameters():
    machineValues = GetMachineValues()
    vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
    importantMachineValues = GetImportantMachineValues(machineValues)
    machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)

In [2]:
!pip install nashpy

In [1]:
import nashpy as nash

In [ ]:
Ei =0.5
simulation_number = 50000
for s in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
    for a in range(1, 11, 1):
        shape = (3,2) #матрица игры
        Attacker = np.zeros(shape=shape)
        Defender = np.zeros(shape=shape) 
        for strategyA in (1, 2):
            for strategyD in (1, 2, 3):
#                 for Ei in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
                    if strategyA == 1 and strategyD == 1:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(simulation_number)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPH(machineArray, importantMachineValues, Ei)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
                                utilityAttacker += UtilityAttacker(machineValues, machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
#                         print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
#                         print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 1 and strategyD == 2:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(simulation_number)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPP(machineArray, Ei)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
                                utilityAttacker += UtilityAttacker(machineValues, machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
#                         print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
#                         print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 1 and strategyD == 3:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(simulation_number)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPN(machineArray)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
                                utilityAttacker += UtilityAttacker(machineValues, machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
#                         print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
#                         print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 1:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(simulation_number)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyHP(machineArray, Ei, importantMachineValues)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
                                utilityAttacker += UtilityAttacker(machineValues, machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
#                         print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
#                         print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 2:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(simulation_number)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparsionStrategyHH(machineArray, Ei)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
                                utilityAttacker += UtilityAttacker(machineValues, machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
#                         print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
#                         print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 3:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(simulation_number)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparsionStrategyHN(machineArray, importantMachineValues)
                            if tmp == True:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
                                utilityAttacker += UtilityAttacker(machineValues, machineArray)
                            else:
                                utilityDefender += UtilityDefender(s, a, Ei, machineValues, machineArray)
#                         print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
#                         print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)

                    Attacker[strategyD-1, strategyA-1] = utilityAttacker
                    Defender[strategyD-1, strategyA-1] = utilityDefender
        print('for s =', s,'a =', a)
        print("Утилиты атакующего ",Attacker)
        print("Утилиты обороняющегося ",Defender)
        game = nash.Game(Attacker, Defender)
        equlibrium = list(game.support_enumeration())[0]
        print("equlibrium", equlibrium )
        print("оптимальная стратегия оброняющегося номер ", np.argmax(equlibrium[0])+1)
        print("оптимальная стратегия атакующего номер ", np.argmax(equlibrium[1])+1)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

...

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

for s = 0.1 a = 2
Утилиты атакующего  [[ 6574.41     0.  ]
 [21092.65     0.  ]
 [    0.       0.  ]]
Утилиты обороняющегося  [[-6.75790482e+03 -4.74210000e-01]
 [-2.10931255e+04 -4.74344000e-01]
 [-4.74760000e-01 -4.75507000e-01]]
equlibrium (array([1., 0., 0.]), array([0., 1.]))
оптимальная стратегия оброняющегося номер  1
оптимальная стратегия атакующего номер  2


  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]